## ASR Metrics references librairies

In [11]:
!pip install --upgrade nltk evaluate tokenizers seqeval sequence-evaluate sentence-transformers rouge jiwer werpy

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached fsspec-2024.6.1-py3-none-any.whl.metadata (11 kB)
Using cached fsspec-2024.6.1-py3-none-any.whl (177 kB)
  Created wheel for werpy: filename=werpy-2.1.2-cp310-cp310-macosx_14_0_arm64.whl size=39538 sha256=0b86a1d593780fe3a1f28cc67f166fc0be9f78a5f882fddeb2158cd2d5c4b206
  Stored in directory: /Users/julienmiquel/Library/Caches/pip/wheels/99/53/3f/1433a81d10ed3aec64cb575b2428810723357f02be8f742e9d
Successfully built werpy
    extract-msg (<=0.29.*)
                 ~~~~~~~^
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.9.0
    Uninstalling fsspec-2024.9.0:
      Successfully uninstalled fsspec-2024.9.0
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 3.0.1
    Uninstalling sentence-transformers-3.0.1:
      Successful

In [9]:
# Simple WER compute
from jiwer import wer

reference = "hello world"
hypothesis = "hello duck"

error = wer(reference, hypothesis)

print(error)

0.5


In [10]:
import jiwer


def process_words_list(ground_truth, generated_text):
  out = jiwer.process_words(
      ground_truth,
      generated_text,
  )

  print(jiwer.visualize_alignment(out))

process_words_list(reference, hypothesis)  

sentence 1
REF: hello world
HYP: hello  duck
               S

number of sentences: 1
substitutions=1 deletions=0 insertions=0 hits=1

mer=50.00%
wil=75.00%
wip=25.00%
wer=50.00%



# Compute wer and other usefull metrics

In [ ]:

from seq_eval import SeqEval
import evaluate

wer_metric = evaluate.load("wer")

evaluator = SeqEval()

def evaluate_data(predictions, references, verbose=False, cleanup = False):
    if cleanup:
        references = [x for x in references if x!= '']
        predictions = [x for x in predictions if x!= '']

    if len(references)!= len(predictions):

        min_arr = min(len(references), len(references))
        print(f"Reduce size to {min_arr}")
        predictions = predictions[0:min_arr]
        references = references[0:min_arr]


    scores = evaluator.evaluate(predictions, references, verbose=verbose)
    if verbose: print(scores)

    wer = wer_metric.compute(references=references, predictions=predictions)
    wer = round(100 * wer, 2)
    print("WER:", wer ,end='\n')
    print("semantic_textual_similarity:",scores['semantic_textual_similarity'],end='\n')
    return wer, scores['semantic_textual_similarity']

evaluate_data(["Hello world"], ["Hello duck"])


In [18]:
import werpy

#Example 1 - Normalize a list of text

input_data = ["It's very popular in Antarctica.","The Sugar Bear character"]
reference = werpy.normalize(input_data)
print(reference)



['its very popular in antarctica', 'the sugar bear character']


In [19]:
input_data = ["It's very popular in Antarctica.","it is  very popular in antarctica"]
reference = werpy.normalize(input_data)

print("Before normalization")
process_words_list([input_data[0]], [input_data[1]])

print("After normalization")
#evaluate_data([reference[0]], [reference[1]])

process_words_list([reference[0]], [reference[1]])


Before normalization
sentence 1
REF: ** It's very popular in Antarctica.
HYP: it   is very popular in  antarctica
      I    S                           S

number of sentences: 1
substitutions=2 deletions=0 insertions=1 hits=3

mer=50.00%
wil=70.00%
wip=30.00%
wer=60.00%

After normalization
sentence 1
REF: ** its very popular in antarctica
HYP: it  is very popular in antarctica
      I   S                           

number of sentences: 1
substitutions=1 deletions=0 insertions=1 hits=4

mer=33.33%
wil=46.67%
wip=53.33%
wer=40.00%

